In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

df = pd.read_csv("input/smart_bin.csv",
    parse_dates=['detected_at'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#Feature Engineering
def feature_engineering(df, i, row):
    detected_at = df.at[i, 'detected_at']#pd.datetime.strptime(row['detected_at'], '%Y-%m-%d %H:%M:%S')
    #df.at[i, 'detected_at'] = detected_at
    df.at[i, 'to_visit'] = 1 if row["bin_level"] >= 3 or row["occluded"] == 2 else 0
    df.at[i, 'hour'] = detected_at.hour
    df.at[i, 'date'] = int(detected_at.strftime('%Y%m%d'))
    return row
    
# DATA PREPARATION - PREPROCESSING

print("Loading the dataset...")

print("Total bins", len(df.bin_serial.unique()))
#
# Feature engineering
#

print("Starting feature engineering...")

default = [0]*len(df)
df = df.assign(**{
    'to_visit': default,
    'date': default,
    'hour': default
})

for index, row in df.iterrows():
    row = feature_engineering(df, index, row)

print("Splitting the dataset...")
    
#
# FIND THE MOST REPRESENTATIVE DAY IN THE DATASET
#

grouped_datebins = df.groupby(["date", "bin_serial"]).size().reset_index(name='counts').sort_values(by=["counts"])
grouped_datebins

grouped_dates = grouped_datebins.groupby(["date"]).size().reset_index(name='counts').sort_values(by=["counts"], ascending=True)
grouped_dates.head() #THE FIRST RECORD IS THE DAY WITH THE MOST OBSERVED BINS

#replace grouped_dates.head(n=1)["date"].values[0] with YYYYmmdd
print("Minimum bins observed by a date: " + str(grouped_dates.head(n=1)["counts"]))
representative_date = grouped_dates.head(n=1)["date"].values[0]
reprdate = df[df["date"] == representative_date] # <----- this is the day dataset with most observed bins

#TAKE THE FIRST VISIT DATASET

reprdate_less_10 = reprdate[reprdate["hour"] <= 10].groupby("bin_serial").last()
first_visit_ds = reprdate_less_10[
    (reprdate_less_10["bin_level"] >= 3) | (reprdate_less_10["occluded"] == 2)
]
first_visit_ds.reset_index(level=0, inplace=True)
first_visit = first_visit_ds.bin_serial.unique()

print(" - Total bins in first visit", len(first_visit))

#TAKE THE SECOND VISIT DATASET

grouped_bybins = df.groupby(["bin_serial", "Latitudine", "Longitudine"]).size().reset_index(name='counts').sort_values(by=["counts"])
second_visit = grouped_bybins[~grouped_bybins.bin_serial.isin(first_visit)]

print(" - Total bins in second visit", len(second_visit))

print("Saving results")


output_table_1 = first_visit_ds.copy()
output_table_2 = second_visit.copy()

output_table_1.to_csv("output/data_preparation/first_visit." + str(representative_date) + ".csv")
output_table_1.to_csv("output/data_preparation/second_visit." + str(representative_date) + ".csv")


Loading the dataset...
Total bins 3147
Starting feature engineering...
Splitting the dataset...
Minimum bins observed by a date: 5    2899
Name: counts, dtype: int64
 - Total bins in first visit 1372
 - Total bins in second visit 1775
Saving results


In [3]:
# BINS LOCATION - Json Format

bybins = df.groupby(["bin_serial", "Latitudine", "Longitudine"]).size().reset_index(name='counts')
bybins = bybins.sort_values(by=["bin_serial"])
bins = "["
for index, row in bybins.iterrows():
    bins += '{"bin_serial": ' + str(row["bin_serial"]) + ', "lat": ' + str(row["Latitudine"]) + ', "lng": ' + str(row["Longitudine"]) + '},'
bins = bins[:-1]
bins += "]"
print(bins)

[{"bin_serial": 119010130.0, "lat": 45.48071229999999, "lng": 9.2252365},{"bin_serial": 119010131.0, "lat": 45.476340140258685, "lng": 9.223657262875236},{"bin_serial": 119010132.0, "lat": 45.47802877384558, "lng": 9.223531596601902},{"bin_serial": 119010133.0, "lat": 45.475771325512106, "lng": 9.223805763673113},{"bin_serial": 119010134.0, "lat": 45.4801975, "lng": 9.2257323},{"bin_serial": 119010135.0, "lat": 45.4801291, "lng": 9.2260746},{"bin_serial": 119010136.0, "lat": 45.4796881449991, "lng": 9.225183088025801},{"bin_serial": 119010137.0, "lat": 45.4755863, "lng": 9.2239964},{"bin_serial": 119010138.0, "lat": 45.4738706429342, "lng": 9.223619657672089},{"bin_serial": 119010139.0, "lat": 45.4800422, "lng": 9.2267372},{"bin_serial": 119010140.0, "lat": 45.47772020000001, "lng": 9.2237316},{"bin_serial": 119010141.0, "lat": 45.47334970007097, "lng": 9.224067013136846},{"bin_serial": 119010142.0, "lat": 45.4801903, "lng": 9.2259397},{"bin_serial": 119010240.0, "lat": 45.515952872957